In [118]:
import requests # Fazer uma solicitação HTTP
from bs4 import BeautifulSoup # Para a manipulação de HTML
import pandas as pd # Criação de dataframes
from io import StringIO # manipular ou ler e escrever strings como se fossem um arquivo.

## Requisitando as Praças de pedágio e colocando em uma lista com seu respectivo link

In [119]:
url_pracas = "https://www.gov.br/antt/pt-br/assuntos/rodovias/pedagio"

resposta = requests.get(url_pracas)

if resposta.status_code == 200:
  soup = BeautifulSoup(resposta.text, 'html.parser')
  pega_titulo = soup.find_all("span", class_="front")
  pega_link = soup.find_all("a", class_="govbr-card-content")


  tabela_pracas = []
  for titulo, link in zip(pega_titulo, pega_link):
      title = titulo.find("span", class_="titulo").text.strip()
      link = link.get('href')
      tabela_pracas.append({"Autopista": title, "Link": link})

  # Removendo alguns titulos irrelevantes que foram captados
  titulos_a_remover = [
                       "SEM PARAR",
                       "ALELO VELOE",
                       "CONECT CAR",
                       "GREENPASS",
                       "Move Mais",
                       "Perguntas Frequentes",
                       "Fale Conosco"
                       ]
  # Iterar e remover os itens
  for item in tabela_pracas[:]:  # Criar uma cópia rasa da lista para iterar
      if item["Autopista"] in titulos_a_remover:
        tabela_pracas.remove(item)
  
  df_tabela_pracas = pd.DataFrame(tabela_pracas)
else:
  print("Erro na requisição")

display(df_tabela_pracas)

,Autopista,Link
0,Autopista Fernão Dias,https://www.gov.br/antt/pt-br/assuntos/rodovia...
1,Autopista Fluminense,https://www.gov.br/antt/pt-br/assuntos/rodovia...
2,Autopista Litoral Sul,https://www.gov.br/antt/pt-br/assuntos/rodovia...
3,Autopista Planalto Sul,https://www.gov.br/antt/pt-br/assuntos/rodovia...
4,Autopista Régis Bittencourt,https://www.gov.br/antt/pt-br/assuntos/rodovia...
5,CONCEBRA,https://www.gov.br/antt/pt-br/assuntos/rodovia...
6,CONCER,https://www.gov.br/antt/pt-br/assuntos/rodovia...
7,CRT (Contrato Encerrado),https://www.gov.br/antt/pt-br/assuntos/rodovia...
8,ECO050 (antiga MGO Rodovias),https://www.gov.br/antt/pt-br/assuntos/rodovia...
9,ECO101,https://www.gov.br/antt/pt-br/assuntos/rodovia...


## Entrando em uma Praça e extraindo as informações


In [120]:
dados_pracas_pedagio = []

todos_titulos = set()

# Itera sobre cada link
for url in tabela_pracas:
    url_destino = url["Link"]
    print(url_destino)
    resposta = requests.get(url_destino)

    if resposta.status_code == 200:
        soup = BeautifulSoup(resposta.text, 'html.parser')
        info_praca = soup.find_all("p")
        
        # Listas para armazenar os títulos e informações
        tabela_praca_destino_titulo = []
        tabela_praca_destino_info = []

        for item in info_praca:
            strong_tag = item.find("strong")
            if strong_tag:
                titulo = strong_tag.get_text(strip=True).replace(":", "")
                texto_item = item.get_text(strip=True)
                
                if ":" in texto_item:
                    # Pega apenas o texto após o título e os dois pontos
                    info = texto_item.split(":", 1)[1].strip()
                else:
                    # Pega o texto após o título
                    info = texto_item.replace(strong_tag.get_text(strip=True), "").strip()

                tabela_praca_destino_titulo.append(titulo)
                tabela_praca_destino_info.append(info)
                todos_titulos.add(titulo)  # Adiciona o título ao conjunto de todos os títulos
            else:
                print(f"Tag <strong> não encontrada em: {item}")

        # Criar a tabela final para este link
        tabela_final_praca_destino = dict(zip(tabela_praca_destino_titulo, tabela_praca_destino_info))
        
        # Adiciona o link como a primeira chave-valor do dicionário
        tabela_final_praca_destino["Link"] = url_destino

        # Adiciona o dicionário à lista
        dados_pracas_pedagio.append(tabela_final_praca_destino)
    else:
        print(f"Erro na requisição para {url_destino}")

# Cria um DataFrame final a partir da lista de tabelas, com todas as colunas possíveis
df_final = pd.DataFrame(dados_pracas_pedagio)

# Reorganiza o DataFrame para garantir que "Link" seja a primeira coluna
colunas = ["Link"] + [col for col in df_final.columns if col != "Link"]
df_final = df_final[colunas]

# Mostra o DataFrame final
display(df_final)


https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/autopista-fernao-dias
https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/autopista-fluminense
https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/autopista-litoral-sul
https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/autopista-planalto-sul
https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/autopista-regis-bittencourt
https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/concebra
https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/concer
https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/contratos-encerrados/crt
https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/eco050
https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concess

,Link,Empresas Controladoras,Trecho Objeto da Concessão,Extensão do Trecho Concedido,Data da Assinatura,Início da Concessão,Prazo da Concessão,Início da Cobrança de Pedágio,Compromissos assumidos,Data de reajuste anual da tarifa de pedágio,Contrato referente ao Edital de nº 001/2008,Telefone de Atendimento,Início da cobrança
0,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ARTERIS S/A,BR-381/MG/SP - Belo Horizonte - São Paulo,562 Km,14/02/2008,18/02/2008,25 anos,19/12/2008,Contrato,19/dez,NaN,NaN,NaN
1,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ARTERIS S/A,BR-101/RJ – Divisa RJ/ES – Ponte Presidente Co...,320 Km,14/02/2008,18/02/2008,25 anos,02/02/2009,Contrato,02/fev,NaN,NaN,NaN
2,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ARTERIS S/A,BR-116/376/PR e BR-101/SC – Trecho Curitiba – ...,"405,94 Km",14/02/2008,18/02/2008,25 anos,22/02/2009,Contrato,22/fev,NaN,NaN,NaN
3,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ARTERIS S/A,BR-116/PR/SC – Curitiba – Divisa SC/RS,413 Km,14/02/2008,18/02/2008,NaN,19/12/2008,Contrato,19/dez,NaN,NaN,NaN
4,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ARTERIS S/A,BR-116/SP/PR – São Paulo – Curitiba,402 Km,14/02/2008,18/02/2008,25 anos,29/12/2008,Contrato,29/dez,NaN,NaN,NaN
5,https://www.gov.br/antt/pt-br/assuntos/rodovia...,TPI – Triunfo Participações e Investimentos S. A.,"BR 060, BR 153 e BR 262 DF / GO / MG","1.176,5 Km",31/01/2014,05/03/2014,30 anos,27/06/2015,NaN,NaN,NaN,NaN,NaN
6,https://www.gov.br/antt/pt-br/assuntos/rodovia...,"Construtora Triunfo, Construcap, Triunfo, CCPS...",BR 040/MG/RJ - trecho Juiz de Fora/MG - Rio de...,180 Km,31/10/1995,01/03/1996,25 anos,20/08/1996,Contrato,20/ago,NaN,NaN,NaN
7,https://www.gov.br/antt/pt-br/assuntos/rodovia...,Investimentos e Participações em Infra-Estrutu...,"BR-116/RJ, trecho Além Paraíba - Teresópolis –...","142,5 Km",22/11/1995,22/03/1996,25 anos,02/09/1996,Contrato PG-156/95-00,02/set,NaN,NaN,NaN
8,https://www.gov.br/antt/pt-br/assuntos/rodovia...,Ecorodovias Infraestrutura e Logística S.A.,NaN,"436,6 Km",05/12/2013,08/01/2014,30 anos,12/04/2015,NaN,NaN,NaN,NaN,NaN
9,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ECORODOVIAS INFRAESTRUTURA E LOGÍSTICA S/A e S...,BR 101/ES/BA - Entroncamento BA-698 (acesso a ...,"475,9 Km",17/04/2013,10/05/2013,25 anos,18/05/2014,Contrato,18/05,NaN,NaN,NaN


### Concatenando todos os dataframes em uma só onde o Link é a coluna em comum para garantir a integridade

In [121]:
df_pracas_dados_merge = pd.merge(df_tabela_pracas, df_final, on="Link", how="inner")

display(df_pracas_dados_merge)

,Autopista,Link,Empresas Controladoras,Trecho Objeto da Concessão,Extensão do Trecho Concedido,Data da Assinatura,Início da Concessão,Prazo da Concessão,Início da Cobrança de Pedágio,Compromissos assumidos,Data de reajuste anual da tarifa de pedágio,Contrato referente ao Edital de nº 001/2008,Telefone de Atendimento,Início da cobrança
0,Autopista Fernão Dias,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ARTERIS S/A,BR-381/MG/SP - Belo Horizonte - São Paulo,562 Km,14/02/2008,18/02/2008,25 anos,19/12/2008,Contrato,19/dez,NaN,NaN,NaN
1,Autopista Fluminense,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ARTERIS S/A,BR-101/RJ – Divisa RJ/ES – Ponte Presidente Co...,320 Km,14/02/2008,18/02/2008,25 anos,02/02/2009,Contrato,02/fev,NaN,NaN,NaN
2,Autopista Litoral Sul,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ARTERIS S/A,BR-116/376/PR e BR-101/SC – Trecho Curitiba – ...,"405,94 Km",14/02/2008,18/02/2008,25 anos,22/02/2009,Contrato,22/fev,NaN,NaN,NaN
3,Autopista Planalto Sul,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ARTERIS S/A,BR-116/PR/SC – Curitiba – Divisa SC/RS,413 Km,14/02/2008,18/02/2008,NaN,19/12/2008,Contrato,19/dez,NaN,NaN,NaN
4,Autopista Régis Bittencourt,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ARTERIS S/A,BR-116/SP/PR – São Paulo – Curitiba,402 Km,14/02/2008,18/02/2008,25 anos,29/12/2008,Contrato,29/dez,NaN,NaN,NaN
5,CONCEBRA,https://www.gov.br/antt/pt-br/assuntos/rodovia...,TPI – Triunfo Participações e Investimentos S. A.,"BR 060, BR 153 e BR 262 DF / GO / MG","1.176,5 Km",31/01/2014,05/03/2014,30 anos,27/06/2015,NaN,NaN,NaN,NaN,NaN
6,CONCER,https://www.gov.br/antt/pt-br/assuntos/rodovia...,"Construtora Triunfo, Construcap, Triunfo, CCPS...",BR 040/MG/RJ - trecho Juiz de Fora/MG - Rio de...,180 Km,31/10/1995,01/03/1996,25 anos,20/08/1996,Contrato,20/ago,NaN,NaN,NaN
7,CRT (Contrato Encerrado),https://www.gov.br/antt/pt-br/assuntos/rodovia...,Investimentos e Participações em Infra-Estrutu...,"BR-116/RJ, trecho Além Paraíba - Teresópolis –...","142,5 Km",22/11/1995,22/03/1996,25 anos,02/09/1996,Contrato PG-156/95-00,02/set,NaN,NaN,NaN
8,ECO050 (antiga MGO Rodovias),https://www.gov.br/antt/pt-br/assuntos/rodovia...,Ecorodovias Infraestrutura e Logística S.A.,NaN,"436,6 Km",05/12/2013,08/01/2014,30 anos,12/04/2015,NaN,NaN,NaN,NaN,NaN
9,ECO101,https://www.gov.br/antt/pt-br/assuntos/rodovia...,ECORODOVIAS INFRAESTRUTURA E LOGÍSTICA S/A e S...,BR 101/ES/BA - Entroncamento BA-698 (acesso a ...,"475,9 Km",17/04/2013,10/05/2013,25 anos,18/05/2014,Contrato,18/05,NaN,NaN,NaN


### Concatenando os links com 'tarifas-de-pedagio'

In [122]:
palavra = 'tarifas-de-pedagio'
url_valores_pedagio = []
for link in df_pracas_dados_merge['Link']:
  link_pedagio = f'{link}/{palavra}'
  url_valores_pedagio.append(link_pedagio)
  
df = pd.DataFrame(url_valores_pedagio, columns=['url_valores_pedagio'])
display(df)

,url_valores_pedagio
0,https://www.gov.br/antt/pt-br/assuntos/rodovia...
1,https://www.gov.br/antt/pt-br/assuntos/rodovia...
2,https://www.gov.br/antt/pt-br/assuntos/rodovia...
3,https://www.gov.br/antt/pt-br/assuntos/rodovia...
4,https://www.gov.br/antt/pt-br/assuntos/rodovia...
5,https://www.gov.br/antt/pt-br/assuntos/rodovia...
6,https://www.gov.br/antt/pt-br/assuntos/rodovia...
7,https://www.gov.br/antt/pt-br/assuntos/rodovia...
8,https://www.gov.br/antt/pt-br/assuntos/rodovia...
9,https://www.gov.br/antt/pt-br/assuntos/rodovia...


## FUNÇÕES 

In [123]:
def checar_qtde_tabelas(url_valores):
    # Faz a requisição para obter o conteúdo da página
    resposta = requests.get(url_valores)

    if resposta.status_code == 200:
        # Cria um objeto BeautifulSoup para parsear o HTML
        soup = BeautifulSoup(resposta.text, 'html.parser')

        # Encontrando a div com id 'parent-fieldname-text'
        div_content = soup.find('div', id='parent-fieldname-text')

        if div_content:
            # Encontra todas as tabelas dentro da div
            tabelas = div_content.find_all("table")
            
            # Checa a quantidade de tabelas encontradas
            if len(tabelas) == 1:
                return '1'
            else:
                return '+1'
        else:
            print("Div com o id 'parent-fieldname-text' não encontrada.")
            return False
    else:
        print(f"Erro ao acessar a página: Status code {resposta.status_code}")
        return False

In [124]:
def extrair_tabela(url):
    # Faz a requisição para obter o conteúdo da página
    resposta = requests.get(url)

    if resposta.status_code == 200:
        # Cria um objeto BeautifulSoup para parsear o HTML
        soup = BeautifulSoup(resposta.text, 'html.parser')
        
        # Encontra a primeira tabela na página
        tabela = soup.find("table")
        
        if tabela:
            # Extrai as linhas da tabela
            linhas = tabela.find_all("tr")
            
            # Inicializa uma lista para armazenar os dados
            dados = []
            
            # Itera sobre as linhas da tabela
            for linha in linhas:
                # Extrai as células de cada linha
                celulas = linha.find_all("td")
                # Extrai o texto de cada célula e adiciona à lista 'dados'
                dados.append([celula.get_text(strip=True) for celula in celulas])
            
            # Remove linhas vazias
            dados = [linha for linha in dados if any(linha)]

            # Criar o DataFrame com os dados extraídos
            tabela_autopista = pd.DataFrame(dados)
            
            
            return tabela_autopista
        else:
            print("Tabela não encontrada na página.")
            return None
    else:
        print(f"Falha ao acessar a página: {resposta.status_code}")
        return None


In [139]:

def processar_tabela_praca_unica(tabela_autopista, nome_coluna='pracas_autopista', valor_coluna='praça única'):
    # GERALMENTE É O FORMATO MAIS SIMPLES DE TABELA, SEM TITULOS ACIMA E COM PRAÇA ÚNICA
    """
    Função para processar uma tabela do DataFrame, ajustando os títulos das colunas e adicionando uma nova coluna.

    Args:
    tabela_autopista (pd.DataFrame): DataFrame a ser processado.
    nome_coluna (str): Nome da nova coluna a ser adicionada. Padrão é 'pracas_autopista'.
    valor_coluna (str): Valor a ser atribuído à nova coluna. Padrão é 'praça única'.

    Returns:
    pd.DataFrame: DataFrame processado com os títulos das colunas ajustados e a nova coluna adicionada.
    """
    # Verificar se a tabela tem pelo menos uma linha e coluna
    if len(tabela_autopista) > 1 and len(tabela_autopista.columns) > 0:
        # Definir a primeira linha como os títulos das colunas
        tabela_autopista.columns = tabela_autopista.iloc[0]
        # Remover a primeira linha (que foi usada como título)
        tabela_autopista = tabela_autopista[1:].reset_index(drop=True)
        tabela_autopista.columns = ['categoria_de_veiculo', 'tipo_de_veiculo', 'numero_de_eixos', 'rodagem', 'multiplicador_da_tarifa', 'valores_praticados'] 
    else:
        raise ValueError("A tabela não contém linhas suficientes para definir os títulos das colunas.")
    

    # Adicionar uma nova coluna
    tabela_autopista.insert(0, nome_coluna, valor_coluna)

    #  CODIGO PARA FORMATAR VIA SUL
    if tabela_autopista['categoria_de_veiculo'].str.contains('P1, P2, P3, P4, P5, P6, P7').any():
        tabela_autopista = tabela_autopista.dropna().reset_index(drop=True)
        tabela_autopista['pracas_autopista'] = 0
        valores = ['Três Cachoeiras', 'Santo Antônio da Patrulha', 'Gravataí', 'Montenegro', 'Paverama', 'Fontoura Xavier', 'Victor Graeff']
        pracas_viasul = pd.DataFrame({'pracas_autopista': valores})
        # Lista com valores de pracas_autopista
        valores_pracas = pracas_viasul['pracas_autopista'].tolist()
            # Criar uma lista para armazenar os DataFrames temporários
        resultados = []
            # Repetir o DataFrame para cada valor de pracas_autopista
        for praca in valores_pracas:
                tabela_temp = tabela_autopista.copy()
                tabela_temp['pracas_autopista'] = praca
                resultados.append(tabela_temp)
            # Concatenar todos os DataFrames temporários em um único DataFrame
        tabela_autopista = pd.concat(resultados, ignore_index=True)
    #  CODIGO PARA FORMATAR VIA SUL      

    
    return tabela_autopista


# Exemplo de uso:
# tabela_autopista = pd.read_csv('caminho_para_tabela_autopista.csv')
# tabela_autopista_processada = processar_tabela(tabela_autopista)
# display(tabela_autopista_processada)
        



In [141]:
def extrair_tabela_texto_acima(tabela_autopista, url_valores):
    # Verificar se a tabela tem pelo menos uma linha e coluna
    if len(tabela_autopista) > 1 and len(tabela_autopista.columns) > 0:
        # Definir a primeira linha como os títulos das colunas
        tabela_autopista.columns = tabela_autopista.iloc[0]
        # Remover a primeira linha (que foi usada como título)
        tabela_autopista = tabela_autopista[1:].reset_index(drop=True)
    else:
        raise ValueError("A tabela não contém linhas suficientes para definir os títulos das colunas.")
    
    # Solicitar o conteúdo da URL
    resposta = requests.get(url_valores)
    if resposta.status_code == 200:
        soup = BeautifulSoup(resposta.text, 'html.parser')
        # Encontrar o primeiro <div> com id="parent-fieldname-text"
        div = soup.find("div", id="parent-fieldname-text")
        # Encontrar o primeiro <p> dentro desse <div>
        p_tag = div.find("p")
        # Extrair o texto dentro do <p>
        texto = p_tag.get_text(strip=True)
        # Remover o prefixo "pracas " e o " e" final
        texto_limpo = texto.replace("Praças ", "").replace(" e ", ", ").replace(" E ", ", ").replace("de Pedágio:  ", "")
        # Dividir o texto em uma lista usando a vírgula como delimitador
        lista_pracas = [p.strip() for p in texto_limpo.split(",")]

        resultados = []
        for praca in lista_pracas:
            tabela_temp = tabela_autopista.copy()
            tabela_temp.insert(0, 'pracas_autopista', praca)
            resultados.append(tabela_temp)

        # Concatenar todos os DataFrames em um só
        tabela_autopista = pd.concat(resultados, ignore_index=True)

        #   CODIGO PARA FORMATAR REGIS BITTENCOURT
        # Adicionar a validação para a string "de Pedágio P1 a P6"
        if tabela_autopista['pracas_autopista'].str.contains('de Pedágio P1 a P6').any():
            # Usar str.replace() para remover 'de Pedágio ' e manter 'P1 a P6'
            tabela_autopista['pracas_autopista'] = 0
            valores = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
            for valor in valores:
                pracas_regis = pd.DataFrame({'pracas_autopista': valores})
            # Lista com valores de pracas_autopista
            valores_pracas = pracas_regis['pracas_autopista'].tolist()
            # Criar uma lista para armazenar os DataFrames temporários
            resultados = []
            # Repetir o DataFrame para cada valor de pracas_autopista
            for praca in valores_pracas:
                tabela_temp = tabela_autopista.copy()
                tabela_temp['pracas_autopista'] = praca
                resultados.append(tabela_temp)
            # Concatenar todos os DataFrames temporários em um único DataFrame
            tabela_autopista = pd.concat(resultados, ignore_index=True)
         #  CODIGO PARA FORMATAR REGIS BITTENCOURT

        #   CODIGO PARA FORMATAR ECOVIAS DO CERRADO
        # Adicionar a validação para a string "de Pedágio P1 a P6"
        if tabela_autopista['pracas_autopista'].str.contains('P1 a P11').any():
            # Usar str.replace() para remover 'de Pedágio ' e manter 'P1 a P6'
            tabela_autopista['pracas_autopista'] = 0
            valores = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11']
            pracas_eco_cerrado = pd.DataFrame({'pracas_autopista': valores})
            # Lista com valores de pracas_autopista
            valores_pracas = pracas_eco_cerrado['pracas_autopista'].tolist()
            # Criar uma lista para armazenar os DataFrames temporários
            resultados = []
            # Repetir o DataFrame para cada valor de pracas_autopista
            for praca in valores_pracas:
                tabela_temp = tabela_autopista.copy()
                tabela_temp['pracas_autopista'] = praca
                resultados.append(tabela_temp)
            # Concatenar todos os DataFrames temporários em um único DataFrame
            tabela_autopista = pd.concat(resultados, ignore_index=True)
         #  CODIGO PARA FORMATAR ECOVIAS DO CERRADO

          
        # Verificar o número de colunas no DataFrame
        num_colunas = len(tabela_autopista.columns)
        print(f"Número de colunas no DataFrame: {num_colunas}")  # Depuração

        # Verifique se o número de colunas corresponde ao número de nomes que você quer dar
        if num_colunas == 7:  # Supondo que você espera 7 colunas (incluindo 'pracas_autopista')
            # Renomear as colunas
            tabela_autopista.columns = ['pracas_autopista', 'categoria_de_veiculo', 'tipo_de_veiculo', 'numero_de_eixos', 'rodagem', 'multiplicador_da_tarifa', 'valores_praticados']
    
    else:
        raise ConnectionError("Erro ao acessar a URL fornecida.")

    return tabela_autopista

In [142]:
url_valores = "https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/ecovias-do-cerrado/tarifas-de-pedagio"
qtde_tabelas = checar_qtde_tabelas(url_valores)
if qtde_tabelas == '1':
  tabela_autopista = extrair_tabela(url_valores)
  tabela_autopista = extrair_tabela_texto_acima(tabela_autopista, url_valores)
else:
  tabela_autopista = extrair_varias_tabelas(url_valores)
############################################################################################################################################

display(tabela_autopista)



Número de colunas no DataFrame: 7


,pracas_autopista,categoria_de_veiculo,tipo_de_veiculo,numero_de_eixos,rodagem,multiplicador_da_tarifa,valores_praticados
0,P1,1,"Automóvel, caminhonete e furgão",2,Simples,"1,0","5,70"
1,P1,2,"Caminhão leve, ônibus, caminhão trator e furgão",2,Dupla,"2,0","11,40"
2,P1,3,Automóvel e caminhonete com semi-reboque,3,Simples,"1,5","8,55"
3,P1,4,"Caminhão, caminhão-trator, caminhão-trator com...",3,Dupla,"3,0","17,10"
4,P1,5,Automóvel e caminhonete com reboque,4,Simples,"2,0","11,40"
...,...,...,...,...,...,...,...
105,P11,6,Caminhão com reboque e caminhão trator com sem...,4,Dupla,"4,0","22,80"
106,P11,7,Caminhão com reboque e caminhão trator com sem...,5,Dupla,"5,0","28,50"
107,P11,8,Caminhão com reboque e caminhão trator com sem...,6,Dupla,"6,0","34,20"
108,P11,9,"Motocicletas, motonetas, triciclos e bicicleta...",2,Simples,"0,5","2,85"


# NÃO MEXER DAQUI PARA CIMA

## Várias praças mas com o mesmo preço (FAZENDO)

In [127]:
url_valores = ["https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/autopista-litoral-sul/tarifas-de-pedagios",
               "https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/autopista-regis-bittencourt/tarifas-de-pedagio",
               
               "https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/viasul/tarifas-de-pedagio",
               "https://www.gov.br/antt/pt-br/assuntos/rodovias/concessionarias/lista-de-concessoes/ecovias-do-cerrado/tarifas-de-pedagio"
              ]

# Dicionário de mapeamento de links para nomes de autopistas
mapa_links_autopistas = dict(zip(df_tabela_pracas['Link'], df_tabela_pracas['Autopista']))

# Inicializar uma lista para armazenar os DataFrames
lista_tabelas = []

for url in url_valores:
    qtde_tabelas = checar_qtde_tabelas(url)
    if qtde_tabelas == '1':
        tabela_autopista = extrair_tabela(url)
        tabela_autopista = extrair_tabela_texto_acima(tabela_autopista, url)
        
        # Adiciona o nome da autopista correspondente ao DataFrame
        nome_autopista = mapa_links_autopistas.get(url.rsplit('/', 1)[0])  # Ignora a parte final do link para comparar
        tabela_autopista['Autopista'] = nome_autopista
        
        lista_tabelas.append(tabela_autopista)  # Adiciona o DataFrame à lista
    else:
        tabela_autopista = extrair_varias_tabelas(url)
        
        # Adiciona o nome da autopista correspondente ao DataFrame
        nome_autopista = mapa_links_autopistas.get(url.rsplit('/', 1)[0])  # Ignora a parte final do link para comparar
        tabela_autopista['Autopista'] = nome_autopista
        
        lista_tabelas.append(tabela_autopista)

# Concatenar todas as tabelas em um único DataFrame
pracas_unicas_df = pd.concat(lista_tabelas, axis=0, ignore_index=True)

# Reorganizar as colunas para deixar 'Autopista' como a primeira coluna
colunas_ordenadas = ['Autopista'] + [col for col in pracas_unicas_df.columns if col != 'Autopista']
pracas_unicas_df = pracas_unicas_df[colunas_ordenadas]

# Exibir o DataFrame final
display(pracas_unicas_df)


Número de colunas no DataFrame: 7
Número de colunas no DataFrame: 7
Número de colunas no DataFrame: 7
Número de colunas no DataFrame: 7


,Autopista,pracas_autopista,categoria_de_veiculo,tipo_de_veiculo,numero_de_eixos,rodagem,multiplicador_da_tarifa,valores_praticados
0,Autopista Litoral Sul,P1,1,"Automóvel, caminhonete e furgão",2,Simples,1,"4,90"
1,Autopista Litoral Sul,P1,2,"Caminhão leve, Ônibus, caminhão-trator e furgão",2,Dupla,2,"9,80"
2,Autopista Litoral Sul,P1,3,Automóvel e caminhonete com semirreboque,3,Simples,"1,5","7,35"
3,Autopista Litoral Sul,P1,4,"Caminhão, caminhão-trator, caminhão-trator com...",3,Dupla,3,"14,70"
4,Autopista Litoral Sul,P1,5,Automóvel e caminhonete com reboque,4,Simples,2,"9,80"
...,...,...,...,...,...,...,...,...
121,Ecovias do Cerrado,P1 a P11,6,Caminhão com reboque e caminhão trator com sem...,4,Dupla,"4,0","22,80"
122,Ecovias do Cerrado,P1 a P11,7,Caminhão com reboque e caminhão trator com sem...,5,Dupla,"5,0","28,50"
123,Ecovias do Cerrado,P1 a P11,8,Caminhão com reboque e caminhão trator com sem...,6,Dupla,"6,0","34,20"
124,Ecovias do Cerrado,P1 a P11,9,"Motocicletas, motonetas, triciclos e bicicleta...",2,Simples,"0,5","2,85"
